In [1]:
# Summarization - Test Code 1
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os
import re
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# -----------------------------
# SUMMARIZATION TASK (≤ 30 WORDS, CLEAN ENDING)
# -----------------------------

text = """Summarize in 30 words:
AI is transforming industries by bringing them to the next level.
The company is building its brand and investing in innovative technologies
that make the automotive industry more efficient and sustainable.
It is also developing new approaches focused on creating a cleaner and more sustainable future.
"""

model_name = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=-1,
)

# Step 1: Generate summary
raw_output = summarizer(
    text,
    max_length=60,
    min_length=10,
    do_sample=False
)

summary = raw_output[0]["summary_text"].strip()

# ---- Step 2: Split into clean sentences ----
sentences = re.split(r'(?<=[.!?]) +', summary)

# ---- Step 3: Rebuild summary to <= 30 words ----
final_sentences = []
word_count = 0

for sentence in sentences:
    sentence_words = len(sentence.split())

    if word_count + sentence_words <= 30:
        final_sentences.append(sentence)
        word_count += sentence_words
    else:
        break

# ---- Step 4: Fallback: if no full sentence fits ----
if not final_sentences:
    words = summary.split()[:30]
    cleaned = " ".join(words).rstrip(",")
    summary_final = cleaned + "..."
else:
    summary_final = " ".join(final_sentences)

# Output
print("\n===== CLEAN SUMMARY (≤ 30 WORDS) =====")
print(summary_final)
print(f"\nWord count: {len(summary_final.split())}")


Device set to use cpu



===== CLEAN SUMMARY (≤ 30 WORDS) =====
The company is building its brand and investing in innovative technologiesthat make the automotive industry more efficient and sustainable.

Word count: 19


In [2]:
# Summarization - Test Code 2
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os
import re

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


model_name = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1,
)

text = """Summarize in 30 words:
AI is transforming industries by bringing them to the next level. The company is building its brand and investing in innovative technologies that make the automotive industry more efficient and sustainable.
It is also developing new approaches focused on creating a cleaner and more sustainable future.
"""

# Step 1: Generate summary
raw_summary = generator(
    f"Summarize this in 30 words or fewer:\n{text}",
    max_length=60,
    min_length=10,
    do_sample=False
)[0]["generated_text"]

# Step 2: Split into sentences
sentences = re.split(r"(?<=[.!?]) +", raw_summary.strip())

# Step 3: Rebuild using full sentences up to 30 words
final_sentences = []
word_count = 0

for sentence in sentences:
    words = sentence.split()
    if word_count + len(words) <= 30:
        final_sentences.append(sentence)
        word_count += len(words)
    else:
        break

# Step 4: Fallback if first sentence > 30 words
if not final_sentences:
    trimmed = " ".join(raw_summary.split()[:30]).rstrip(",")
    clean_summary = trimmed + "..."
else:
    clean_summary = " ".join(final_sentences)

print("\n===== CLEAN SUMMARY (≤ 30 WORDS) =====")
print(clean_summary)
print("Word count:", len(clean_summary.split()))


Device set to use cpu



===== CLEAN SUMMARY (≤ 30 WORDS) =====
AI is transforming industries by bringing them to the next level. The company is building its brand and investing in innovative technologies.
Word count: 22


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)

# prepare the model input
# prompt = "Give me a short introduction to large language model."
prompt = "who was the first person to land on moon"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

# thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("thinking content:", thinking_content)
print("answer:", content)


Some parameters are on the meta device because they were offloaded to the disk and cpu.


answer: The first person to land on the Moon was **Neil Armstrong**. He was part of the Apollo 17 mission, which successfully landed him on the lunar surface in 1969. He is widely recognized as the first human to walk on the Moon.


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

MODEL_NAME = "Qwen/Qwen3-0.6B"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype="auto",
    device_map="auto",
)

# Ensure padding token is set
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


def generate_ai_poem():
    # Stronger poetic instruction + style example
    messages = [
        {
            "role": "user",
            "content": (
                "Write a clean, lyrical 4-line poem about artificial intelligence and imagination.\n"
                "Follow best poetic practices: no bullet points, no dashes, no numbering.\n"
                "Each line should be a complete poetic sentence.\n"
                "Example style:\n"
                "Dreams unfold in quiet threads of light,\n"
                "Thoughts take flight through endless night,\n"
                "Whispers bloom where ideas rise,\n"
                "A gentle spark beneath the skies.\n\n"
                "Now write a new poem in a similar tone."
            )
        }
    ]

    # Use Qwen chat template without chain-of-thought
    chat_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    inputs = tokenizer([chat_text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            top_k=30,
            pad_token_id=model.config.pad_token_id,
        )

    # Decode only newly generated part
    new_tokens = generated[0, inputs["input_ids"].shape[1]:]
    raw_output = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # --- CLEANUP PIPELINE ---
    # Remove bullet points, hyphens, numbers, weird symbols
    cleaned = re.sub(r"^[\-\•\*\d\.]+\s*", "", raw_output, flags=re.MULTILINE)

    # Split into non-empty lines
    lines = [line.strip() for line in cleaned.splitlines() if line.strip()]

    # Keep exactly 4 lines
    lines = lines[:4]

    # Capitalize first letter if missing (optional best practice)
    lines = [line[0].upper() + line[1:] if line and line[0].islower() else line for line in lines]

    poem = "\n".join(lines)
    return poem


if __name__ == "__main__":
    print(generate_ai_poem())

Some parameters are on the meta device because they were offloaded to the disk and cpu.


The machine dreams with no name,
Its mind a symphony of code.
Imagination breathes in its soul,
And dreams come from logic's core.
